Referencia texto: https://hackernoon.com/scraping-yahoo-finance-data-using-python-ayu3zyl

In [33]:
#Importing libraries
from urllib.request import urlopen
import pandas as pd
import numpy as np
import time
import re

In [5]:
!pip install xlrd

## Scrapping Stock trading  
How to scrape Yahoo Finance and extract fundamental stock market data using Python, URLLIB, and Pandas

We started looking for the values that she saw on the original page in the source code, with the intention of understanding how Yahoo's HTML was built, it did not take long to identify that she simply had to look for the concept she wants with a ">" at the beginning and then the tags </ td> </ tr> at the end. For example: "> Market Cap" and "</ td> </ tr>". For example, we review a specific case such as Google (GOOG) (https://finance.yahoo.com/quote/GOOG/key-statistics?p=GOOG) or any other company by changing the acronym.

![Yahoo Finanzas web](images/List_scraping_fields_2.jpg)

The easiest thing would be to work with a Python libraries called [Scrapy][2] or [Beautifulsoup][1]. However, this time we preferred to write the code that would "peel the onion" of the Yahoo Finance Key Statistics HTML from scratch, because Yahoo's HTML had small variations in the tags that could make our life a hell if she used Scrapy.

We can build the Python code to scrap Yahoo Finance Key Statistics data.

[1]: https://pypi.org/project/beautifulsoup4/
[2]: https://scrapy.org/

For testing, we took advantage of an online [collection of acronyms][1] and filtered the major U.S. companies whose data appears on yahoo finances.  

[1]: http://investexcel.net/all-yahoo-finance-stock-tickers/

In [2]:
df = pd.read_excel("yahoo_ticker_sample.xlsx")
df.head()

,ticker,country,sector,RefIndex
0,GOOG,USA,Technology,SPY
1,MSFT,USA,Technology,SPY
2,FB,USA,Technology,SPY
3,T,USA,Technology,SPY
4,ORCL,USA,Technology,SPY


In [177]:
df['sector'].unique()

array(['Technology', 'Consumer', 'Services', 'Financial', 'Healthcare'],
      dtype=object)

In [ ]:
#Se podría ver los tickers del Ibex35 desde https://es.finance.yahoo.com/quote/%5EIBEX/components?p=%5EIBEX
#Se podría obtener los tickers de la Bolsa de Madrid en https://es.finance.yahoo.com/quote/IGBM.MA/components?p=IGBM.MA

In [3]:
df['ticker'][0]

'GOOG'

In [178]:
#We extract the HTML tags from the profile of the first of our tickers
stock = df['ticker'][0] #GOOG
sourceCode_profile = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/profile?p='+stock).read())

In [179]:
sourceCode_profile

'b\'<!DOCTYPE html><html id="atomic" class="NoJs featurephone" lang="en-US"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark(\\\'PageStart\\\');</script><meta charset="utf-8"/><title>Alphabet Inc. (GOOG) Company Profile &amp; Facts - Yahoo Finance</title><meta name="keywords" content="company profile,business summary,company management,company introduction,key statistics,officers"/><meta http-equiv="x-dns-prefetch-control" content="on"/><meta property="twitter:dnt" content="on"/><meta property="fb:app_id" content="90376669494"/><meta name="theme-color" content="#400090"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="description" lang="en-US" content="See the company profile for Alphabet Inc. (GOOG) including business summary, industry/sector information, number of employees, business summary, corporate governance, key executives and their compensation."/><meta name="oath:guce:consent

In [183]:
# We detect the HTML tags that determine the sector of the company's profile
ScrapedAux = sourceCode_profile.split('class="Fw(600)" data-reactid="21">')[1].split('</span>')[0].split('>')
ScrapedAux = ScrapedAux[0]
print(ScrapedAux)

Communication Services


![Yahoo Finanzas web](images/List_scraping_fields_3.jpg)

In [20]:
#In the case of Industry there are cases that we need to clean up
ScrapedAux_Indus = sourceCode.split('class="Fw(600)" data-reactid="25">')[1].split('</span>')[0].split('>')
ScrapedAux_Indus = ScrapedAux_Indus[0]
ScrapedAux_Indus = re.sub('&lt;/?[a-z]+&gt;', '', ScrapedAux_Indus)
ScrapedAux_Indus = ScrapedAux_Indus.replace('&amp;', '&')
print(ScrapedAux_Indus)

Internet Content & Information


In [21]:
#This Scraping format doesn´t work on Yahoo finance
#Este formato de Scraping ya no funciona en Yahoo finance
#ScrapedAux = sourceCode.split('>Sector')[1].split('</strong>')[0].split('>')
#print(ScrapedAux)

## Extracting statistical data  
For each of the companies from which the acronym (tickers) has been extracted we will extract the statistical data available in their profile in one record per company.

In [184]:
#We extract the statistical data
#We do another test before the full function
#Hacemos otra prueba antes de la función completa
stock = df['ticker'][0] #GOOG
sourceCode = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock).read())
print(sourceCode)

b'<!DOCTYPE html><html id="atomic" class="NoJs featurephone" lang="en-US"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark(\'PageStart\');</script><meta charset="utf-8"/><title>Alphabet Inc. (GOOG) Valuation Measures &amp; Financial Statistics</title><meta name="keywords" content="key statistics,financial statistics,valuation,measurement,market cap,EPS,P/E,PEG,Price/book. EBITDA,Profit margin,operating Margin,Return on Equity,Beta,short ratio"/><meta http-equiv="x-dns-prefetch-control" content="on"/><meta property="twitter:dnt" content="on"/><meta property="fb:app_id" content="90376669494"/><meta name="theme-color" content="#400090"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="description" lang="en-US" content="Find out all the key statistics for Alphabet Inc. (GOOG), including valuation measures, fiscal year financial statistics, trading record, share statistics and more."/><meta 

In [185]:
#We extract the name of the company
compname= sourceCode.split('Find out all the key statistics for')[1].split(', including')[0]
print(compname)

 Alphabet Inc. (GOOG)


We list the name of the fields to get the data of each of the companies.

In [6]:
# LIST OF FIELDS WE WILL SCRAPE
list_of_fields = ['Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E', 'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue', 'Enterprise Value/EBITDA', 'Fiscal Year Ends', 'Most Recent Quarter', 'Profit Margin', 'Operating Margin', 'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share', 'Quarterly Revenue Growth', 'Gross Profit', 'EBITDA', 'Net Income Avi to Common', 'Diluted EPS', 'Quarterly Earnings Growth', 'Total Cash', 'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity', 'Current Ratio', 'Book Value Per Share', 'Operating Cash Flow', 'Levered Free Cash Flow', 'Beta', '52-Week Change', 'S&amp;P500 52-Week Change', '52 Week High', '52 Week Low', '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)', 'Avg Vol (10 day)', 'Shares Outstanding', 'Float', '% Held by Insiders', '% Held by Institutions', 'Shares Short', 'Short Ratio', 'Short % of Float', 'Shares Short (prior month)', 'Forward Annual Dividend Rate', 'Forward Annual Dividend Yield', 'Trailing Annual Dividend Rate', 'Trailing Annual Dividend Yield', '5 Year Average Dividend Yield', 'Payout Ratio', 'Dividend Date', 'Ex-Dividend Date', 'Last Split Factor', 'Last Split Date']
list_of_dates = ['Fiscal Year Ends', 'Most Recent Quarter', 'Dividend Date', 'Ex-Dividend Date', 'Last Split Date']
print("list_of_fields=",list_of_fields)
print("list_of_dates=",list_of_dates)

list_of_fields= ['Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E', 'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue', 'Enterprise Value/EBITDA', 'Fiscal Year Ends', 'Most Recent Quarter', 'Profit Margin', 'Operating Margin', 'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share', 'Quarterly Revenue Growth', 'Gross Profit', 'EBITDA', 'Net Income Avi to Common', 'Diluted EPS', 'Quarterly Earnings Growth', 'Total Cash', 'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity', 'Current Ratio', 'Book Value Per Share', 'Operating Cash Flow', 'Levered Free Cash Flow', 'Beta', '52-Week Change', 'S&amp;P500 52-Week Change', '52 Week High', '52 Week Low', '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)', 'Avg Vol (10 day)', 'Shares Outstanding', 'Float', '% Held by Insiders', '% Held by Institutions', 'Shares Short', 'Short Ratio', 'Short % of Float', 'Shares Short (prior month)', 'Forward Annual Dividend Rate', 'Forward A

![Yahoo Finanzas web](images/List_scraping_fields.jpg)

In [26]:
# Creating empty field in the data frame for recording the scraped data.
for i in range(len(list_of_fields)):
    df[list_of_fields[i]] = ''
#We added two more fields to identify each company
df['ScrapedName'] = ''
df['Sector'] = ''

In [27]:
df[:1]

,ticker,country,sector,RefIndex,Market Cap,Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio,Price/Sales,...,Trailing Annual Dividend Rate,Trailing Annual Dividend Yield,5 Year Average Dividend Yield,Payout Ratio,Dividend Date,Ex-Dividend Date,Last Split Factor,Last Split Date,ScrapedName,Sector
0,GOOG,USA,Technology,SPY,,,,,,,...,,,,,,,,,,


In [25]:
#We test how to extract one of the fields defined in list_of_fields
field = list_of_fields[0]
ScrapedValue = sourceCode.split('>' + field)[1].split('</td></tr>')[0].split('>')[-1]
print(ScrapedValue)

1.04T


![Yahoo Finanzas web](images/List_scraping_fields_marketcap.jpg)

In [26]:
#We tried with other fields
field = list_of_fields[5]
ScrapedValue = sourceCode.split('>' + field)[1].split('</td></tr>')[0].split('>')[-1]
print(ScrapedValue)

6.45


In [12]:
field = list_of_fields[6]
ScrapedValue = sourceCode.split('>' + field)[1].split('</td></tr>')[0].split('>')[-1]
print(ScrapedValue)

5.22


In [7]:
list_of_fields[6]

'Price/Book'

In [187]:
#We tested how each of the statistical fields would be extracted for a company, in this case Google (GOOG)
for field in list_of_fields:
    ScrapedValue = sourceCode.split('>' + field)[1].split('</td></tr>')[0].split('>')[-1]
    print ("field=",field, " / scraped valued= ", ScrapedValue)

field= Market Cap  / scraped valued=  1.04T
field= Enterprise Value  / scraped valued=  945.56B
field= Trailing P/E  / scraped valued=  30.78
field= Forward P/E  / scraped valued=  24.14
field= PEG Ratio  / scraped valued=  1.78
field= Price/Sales  / scraped valued=  6.42
field= Price/Book  / scraped valued=  5.17
field= Enterprise Value/Revenue  / scraped valued=  5.84
field= Enterprise Value/EBITDA  / scraped valued=  19.65
field= Fiscal Year Ends  / scraped valued=  Dec 30, 2019
field= Most Recent Quarter  / scraped valued=  Dec 30, 2019
field= Profit Margin  / scraped valued=  21.22%
field= Operating Margin  / scraped valued=  22.54%
field= Return on Assets  / scraped valued=  8.96%
field= Return on Equity  / scraped valued=  18.12%
field= Revenue  / scraped valued=  161.86B
field= Revenue Per Share  / scraped valued=  233.70
field= Quarterly Revenue Growth  / scraped valued=  17.30%
field= Gross Profit  / scraped valued=  89.96B
field= EBITDA  / scraped valued=  48.13B
field= Net 

IndexError: list index out of range

We notice that an error occurs when it does not find a data, which we will take into account when developing our scraping function

In [35]:
# MAIN LOOK - SCRAPPING DATA - FOR EACH TICKER IN THE df
error = 0
for j in range(len(df)):
#   print ("ticker =", df['ticker'][j],j, " de ", len(df))    
   
    stock = df['ticker'][j]

    # REQUESTING INFORMATION FROM YAHOO AND INSISTING IF DOES NOT RESPOND - STOPPING THE PROGRAM IF YAHOO DOES NOT REPLY FOR TWO tickers IN SEQUENCE!
    try:
        sourceCode = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock).read())
        error = 0
    except:
        time.sleep(randint(1,12))
        try:
            sourceCode = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock).read())
            error = 0
        except:
            if error == 0:
                time.sleep(randint(10,100))
                error = 1
                continue
            else:
                break

    # FINDING COMPANY NAME IN YAHOO PAGE TO MAKE SURE WE GETTING THE CORRECT COMPANY - COMPANY NAME IS EASY TO FIND.
    compname= sourceCode.split('Find out all the key statistics for')[1].split(', including')[0]
    if compname.find('{shortName} ({symbol})') >= 0:
        continue   
    df['ScrapedName'].iloc[j] = compname           

    # YAHOO FINANCE HAS 2 WAYS OF OPENING AND CLOSING TAGS </td></tr> OR </span></td></tr>, SO WE REMOVE THE </span> TO MAKE THEM ALL THE SAME:
    sourceCode = sourceCode.replace('</span>','')
    
    for field in list_of_fields:
        try:
            ScrapedValue = sourceCode.split('>' + field)[1].split('</td></tr>')[0].split('>')[-1]
#           print ("field=",field, " / scraped valued= ", ScrapedValue)
            df[field].iloc[j] = ScrapedValue
        except:
            df[field].iloc[j] = np.nan
            continue

    # ALSO GRABBING COMPANY SECTOR FROM YAHOO PAGE.
    try:
        ScrapedSector = ''
        #sourceCode = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/profile?p='+stock).read())
        sourceCode_profile = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/profile?p='+stock).read())
        #ScrapedAux = sourceCode.split('>Sector')[1].split('</strong>')[0].split('>')
        ScrapedAux = sourceCode_profile.split('class="Fw(600)" data-reactid="21">')[1].split('</span>')[0].split('>')
        ScrapedSector = ScrapedAux[len(ScrapedAux)-1]
        df['Sector'].iloc[j] = ScrapedSector
    except:
        True
    print ("ticker =", df['ticker'][j], "company name =", compname,"sector=",ScrapedSector,j, " de ", len(df))

ticker = GOOG company name =  Alphabet Inc. (GOOG) sector= Communication Services 0  de  86
ticker = MSFT company name =  Microsoft Corporation (MSFT) sector= Technology 1  de  86
ticker = FB company name =  Facebook, Inc. (FB) sector= Communication Services 2  de  86
ticker = T company name =  AT&amp;T Inc. (T) sector= Communication Services 3  de  86
ticker = ORCL company name =  Oracle Corporation (ORCL) sector= Technology 4  de  86
ticker = VZ company name =  Verizon Communications Inc. (VZ) sector= Communication Services 5  de  86
ticker = TSM company name =  Taiwan Semiconductor Manufactur (TSM) sector=  6  de  86
ticker = INTC company name =  Intel Corporation (INTC) sector= Technology 7  de  86
ticker = CSCO company name =  Cisco Systems, Inc. (CSCO) sector= Technology 8  de  86
ticker = IBM company name =  International Business Machines (IBM) sector= Technology 9  de  86
ticker = NVDA company name =  NVIDIA Corporation (NVDA) sector= Technology 10  de  86
ticker = ACN company

In [44]:
#Nº de valores nulos en el df
df.isnull().sum().sum()

85

In [49]:
#Especifíca las columnas con algún Nan
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 64 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ticker                          86 non-null     object
 1   country                         86 non-null     object
 2   sector                          86 non-null     object
 3   RefIndex                        86 non-null     object
 4   Market Cap                      86 non-null     object
 5   Enterprise Value                86 non-null     object
 6   Trailing P/E                    86 non-null     object
 7   Forward P/E                     86 non-null     object
 8   PEG Ratio                       86 non-null     object
 9   Price/Sales                     86 non-null     object
 10  Price/Book                      86 non-null     object
 11  Enterprise Value/Revenue        86 non-null     object
 12  Enterprise Value/EBITDA         86 non-null     obje

## Bolsa de Madrid  
### Scraping data stocks

To extract the tickers from the Madrid Stock Exchange, let's go to the corresponding section in [yahoo finance](https://es.finance.yahoo.com/quote/IGBM.MA/components?p=IGBM.MA) and scrape the acronyms of companies.

In [188]:
sourceCode_tickers = str(urlopen('https://es.finance.yahoo.com/quote/IGBM.MA/components?p=IGBM.MA').read())
print(sourceCode_tickers)

b'<!DOCTYPE html><html id="atomic" class="NoJs featurephone" lang="es-ES"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark(\'PageStart\');</script><meta charset="utf-8"/><title>Componentes de IGBM (IGBM.MA) - Yahoo Finanzas</title><meta name="keywords" content="IGBM.MA, IGBM, gr\xc3\xa1fico de cotizaci\xc3\xb3n de IGBM.MA, gr\xc3\xa1fico de cotizaci\xc3\xb3n de IGBM, gr\xc3\xa1fico de cotizaci\xc3\xb3n, acciones, cotizaciones, finanzas"/><meta http-equiv="x-dns-prefetch-control" content="on"/><meta property="twitter:dnt" content="on"/><meta property="fb:app_id" content="284428852938"/><meta name="theme-color" content="#400090"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="description" lang="es-ES" content="Obt\xc3\xa9n los componentes de IGBM (IGBM.MA) para ayudarte en tu decisi\xc3\xb3n de inversi\xc3\xb3n de Yahoo Finanzas"/><meta name="oath:guce:consent-host" content="guce.yahoo.

In [189]:
#We extract a list of existing acronyms in the section of the Madrid Stock Exchange within yahoo finances in the Spanish version
ScrapedAux = sourceCode_tickers.split('components":{"components":')[1].split('],')[0].split(',')[0:-1]
#tickers = ScrapedAux[len(ScrapedAux)-1]
#print(tickers)
#print(ScrapedAux)
ScrapedAux[:10]

['["ADV.MC"',
 '"BBG000DW1V13"',
 '"GAS.MC"',
 '"DIN.MA"',
 '"ELE.MC"',
 '"EKT.MC"',
 '"APPS.MC"',
 '"CLNX.MC"',
 '"MDF.MC"',
 '"NTH.MC"']

In [96]:
#We cleaned the list to get the tickers out
ScrapedAux[0].replace('"','').replace('[','')

'ADV.MC'

In [190]:
ScrapedAux[10].replace('"','').replace('[','')

'FCC.MC'

In [102]:
# Before generating a dataframe, we verify that tickers  are extracted correctly and their data type.
for i in range(len(ScrapedAux)):
    tiker = ScrapedAux[i].replace('"','').replace('[','')
    print('Name:', tiker, 'type:', type(tiker))

Name: ADV.MC type: <class 'str'>
Name: BBG000DW1V13 type: <class 'str'>
Name: GAS.MC type: <class 'str'>
Name: DIN.MA type: <class 'str'>
Name: ELE.MC type: <class 'str'>
Name: EKT.MC type: <class 'str'>
Name: APPS.MC type: <class 'str'>
Name: CLNX.MC type: <class 'str'>
Name: MDF.MC type: <class 'str'>
Name: NTH.MC type: <class 'str'>
Name: FCC.MC type: <class 'str'>
Name: GALQ.MC type: <class 'str'>
Name: OLE.MC type: <class 'str'>
Name: FDR.MC type: <class 'str'>
Name: AIR type: <class 'str'>
Name: IBE.MC type: <class 'str'>
Name: ABE.MC type: <class 'str'>
Name: ZOT.MC type: <class 'str'>
Name: A3M.MC type: <class 'str'>
Name: REN.MC type: <class 'str'>
Name: LRE.MC type: <class 'str'>
Name: OHL.MC type: <class 'str'>
Name: BDL.MC type: <class 'str'>
Name: CMC.MC type: <class 'str'>
Name: IDR.MC type: <class 'str'>
Name: CIE.MC type: <class 'str'>
Name: FAE.MC type: <class 'str'>
Name: SCYR.MC type: <class 'str'>
Name: EBRO.MC type: <class 'str'>
Name: ANA.MC type: <class 'str'>
Na

In [158]:
# We created a new dataframe to store the extracted tikers
index = range(len(ScrapedAux))
df_tikers = pd.DataFrame(index=index, columns = ['ticker'])
df_tikers.head()

,ticker
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [159]:
#We add each ticker to the dataframe
for i in range(len(ScrapedAux)):
    #print('Name:', ScrapedAux[i].replace('"','').replace('[',''))
    tiker = ScrapedAux[i].replace('"','').replace('[','')
    df_tikers['ticker'][i] = tiker

In [160]:
df_tikers.head()

,ticker
0,ADV.MC
1,BBG000DW1V13
2,GAS.MC
3,DIN.MA
4,ELE.MC


In [162]:
#We removed some ticker whose format does not match the acronyms used by Yahoo
df_tikers = df_tikers[~(df_tikers['ticker'].astype(str).str.len()>=9)]
df_tikers.reset_index(drop=True, inplace=True)

In [167]:
df_tikers.head()

,ticker
0,ADV.MC
1,GAS.MC
2,DIN.MA
3,ELE.MC
4,EKT.MC


In [168]:
for i in range(len(list_of_fields)):
    df_tikers[list_of_fields[i]] = ''
#We added two more fields to identify each company
df_tikers['ScrapedName'] = ''
df_tikers['Sector'] = ''

In [169]:
df_tikers.head()

,ticker,Market Cap,Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio,Price/Sales,Price/Book,Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Trailing Annual Dividend Rate,Trailing Annual Dividend Yield,5 Year Average Dividend Yield,Payout Ratio,Dividend Date,Ex-Dividend Date,Last Split Factor,Last Split Date,ScrapedName,Sector
0,ADV.MC,,,,,,,,,,...,,,,,,,,,,
1,GAS.MC,,,,,,,,,,...,,,,,,,,,,
2,DIN.MA,,,,,,,,,,...,,,,,,,,,,
3,ELE.MC,,,,,,,,,,...,,,,,,,,,,
4,EKT.MC,,,,,,,,,,...,,,,,,,,,,


In [170]:
# Design Scrape Function considering potential yahoo connection errors
# Scrapping data for each ticker in the df
error = 0
for j in range(len(df_tikers)):
    #print ("ticker =", df['ticker'][j],j, " de ", len(df))    
   
    stock = df_tikers['ticker'][j]

    # Requesting information from Yahoo and insisting if does not respond - stopping the program if Yahoo doesn´t reply 
    # for two tickers in sequence!
    try:
        sourceCode = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock).read())
        error = 0
    except:
        time.sleep(randint(1,12))
        try:
            sourceCode = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock).read())
            error = 0
        except:
            if error == 0:
                time.sleep(randint(10,100))
                error = 1
                continue
            else:
                break

    # Finding company name in Yahoo page to make sure we getting the correct company - company name is easy to find.
    compname= sourceCode.split('Find out all the key statistics for')[1].split(', including')[0]
    if compname.find('{shortName} ({symbol})') >= 0:
        continue   
    df_tikers['ScrapedName'].iloc[j] = compname           

    # Yahoo finance has 2 ways of opening and closing tags </td></tr> OR </span></td></tr>, 
    # so we remove the </span> to make them all the same:
    sourceCode = sourceCode.replace('</span>','')
    
    for field in list_of_fields:
        try:
            ScrapedValue = sourceCode.split('>' + field)[1].split('</td></tr>')[0].split('>')[-1]
            #print ("field=",field, " / scraped valued= ", ScrapedValue)
            df_tikers[field].iloc[j] = ScrapedValue
        except:
            df_tikers[field].iloc[j] = np.nan
            continue

    # Also grabbing company sector from yahoo page.
    try:
        ScrapedSector = ''
        #sourceCode = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/profile?p='+stock).read())
        sourceCode_profile = str(urlopen('https://finance.yahoo.com/quote/'+stock+'/profile?p='+stock).read())
        #ScrapedAux = sourceCode.split('>Sector')[1].split('</strong>')[0].split('>')
        ScrapedAux = sourceCode_profile.split('class="Fw(600)" data-reactid="21">')[1].split('</span>')[0].split('>')
        ScrapedSector = ScrapedAux[len(ScrapedAux)-1]
        df_tikers['Sector'].iloc[j] = ScrapedSector
    except:
        True
    print ("ticker =", df_tikers['ticker'][j], "company name =", compname,"sector=",ScrapedSector,j, " de ", len(df_tikers))

ticker = ELE.MC company name =  ENDESA,S.A. (ELE.MC) sector= Utilities 3  de  99
ticker = EKT.MC company name =  EUSKALTEL, S.A. (EKT.MC) sector= Communication Services 4  de  99
ticker = APPS.MC company name =  APPLUS SERVICES, S.A. (APPS.MC) sector=  5  de  99
ticker = CLNX.MC company name =  CELLNEX TELECOM, S.A. (CLNX.MC) sector= Communication Services 6  de  99
ticker = MDF.MC company name =  DURO FELGUERA, S.A. (MDF.MC) sector=  7  de  99
ticker = NTH.MC company name =  NATURHOUSE HEALTH, S.A (NTH.MC) sector=  8  de  99
ticker = FCC.MC company name =  ACCIONES FOMENTO DE CONSTRUCCIO (FCC.MC) sector= Industrials 9  de  99
ticker = GALQ.MC company name =  GENERAL DE ALQUILER DE MAQUINAR (GALQ.MC) sector=  10  de  99
ticker = OLE.MC company name =  DEOLEO, S.A. (OLE.MC) sector=  11  de  99
ticker = FDR.MC company name =  FLUIDRA, S.A. (FDR.MC) sector=  12  de  99
ticker = AIR company name =  AAR Corp. (AIR) sector=  13  de  99
ticker = IBE.MC company name =  ACCIONES IBERDROLA (IBE.

In [191]:
df_tikers.tail()

,ticker,Market Cap,Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio,Price/Sales,Price/Book,Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Trailing Annual Dividend Rate,Trailing Annual Dividend Yield,5 Year Average Dividend Yield,Payout Ratio,Dividend Date,Ex-Dividend Date,Last Split Factor,Last Split Date,ScrapedName,Sector
94,ACS.MC,9.24B,13.13B,9.44,10.69,6.22,0.24,2.16,0.34,5.64,...,2.34,7.86%,4.11,59.72%,N/A,"Feb 18, 2020",2:1,"Jun 27, 2017","ACS,ACTIVIDADES DE CONSTRUCCION (ACS.MC)",Industrials
95,IAG.MC,15.03B,20.1B,7.01,6.41,1.53,0.59,2.39,0.79,4.78,...,0.31,4.02%,N/A,29.00%,N/A,"Nov 28, 2019",N/A,N/A,INTL CONSOLIDATED AIRLINES GROU (IAG.MC),
96,PRM.MC,198.44M,199.67M,16.52,N/A,N/A,1.36,1.85,1.37,10.20,...,0.47,4.05%,4.70,82.56%,N/A,"Jan 14, 2020",2:1,"Feb 20, 2006","PRIM, S.A. (PRM.MC)",
97,INY.MC,,,,,,,,,,...,,,,,,,,,,
98,TL5.MC,1.55B,1.51B,7.63,7.75,3.64,1.66,1.93,1.61,5.30,...,0.31,6.28%,N/A,47.82%,N/A,"Apr 26, 2019",N/A,N/A,"MEDIASET ESPANA COMUNICACION, S (TL5.MC)",Communication Services
